In [6]:
import numpy as np
import pandas as pd
dataset = pd.read_csv('updated_test.csv')
dataset = dataset[ dataset['YEAR']>1980 ]
dataset = dataset.dropna()
X = dataset.iloc[:,[0,3,4,6,7]].values
y = dataset.iloc[:,5].values
global accuracies
accuracies=[]
df=pd.DataFrame(dataset)
print(df[4000:4005,:])

TypeError: '(slice(4000, 4005, None), slice(None, None, None))' is an invalid key

In [7]:
#data pre-processing
print(X.shape)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
X[:, 1] = labelencoder_X.fit_transform(X[:, 1])
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])
onehotencoder = OneHotEncoder(categorical_features = [0,1,3])
X = onehotencoder.fit_transform(X).toarray()
print(X.shape)
X = X[:,1:]
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

(4187, 5)
(4187, 48)


In [126]:
# example of saving sub-models for later use in a stacking ensemble
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from os import makedirs

# fit model on dataset
def fit_model(trainX, trainy):
    # define model
    classifier = Sequential()
    classifier.add(Dense(units=32, input_dim=47, activation="relu", kernel_initializer="uniform"))
    classifier.add(Dense(units=16, activation="relu", kernel_initializer="uniform"))
    classifier.add(Dense(units=6, activation="softmax", kernel_initializer="uniform"))
    classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 1)
    return  classifier

# generate 2d classification dataset

# create directory for models
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 0)
onehotencoder = OneHotEncoder(categorical_features = [0])
y_train = np.reshape(y_train,(-1,1))
y_train = onehotencoder.fit_transform(y_train).toarray()
# Feature Scaling
from sklearn.preprocessing import StandardScaler
#normalization
#val-mean/n
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
print(X_train.shape)
print(X_test.shape)
makedirs('xyz_models')
# fit and save models
n_members = 5
for i in range(n_members):
	# fit model
	classifier = fit_model(X_train, y_train)
	# save model
	filename = 'xyz_models/model_' + str(i + 1) + '.h5'
	classifier.save(filename)
	print('>Saved %s' % filename)

(3349, 47)
(838, 47)
Epoch 1/1
3349/3349 [==============================] - 5s - loss: 0.5240 - acc: 0.9364     
>Saved xyz_models/model_1.h5
Epoch 1/1
3349/3349 [==============================] - 5s - loss: 0.5109 - acc: 0.9379     
>Saved xyz_models/model_2.h5
Epoch 1/1
3349/3349 [==============================] - 6s - loss: 0.4751 - acc: 0.9391     - ETA: 4s - 
>Saved xyz_models/model_3.h5
Epoch 1/1
3349/3349 [==============================] - 4s - loss: 0.5325 - acc: 0.9376     
>Saved xyz_models/model_4.h5
Epoch 1/1
3349/3349 [==============================] - 7s - loss: 0.4853 - acc: 0.9403     
>Saved xyz_models/model_5.h5


In [133]:
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
dataset = pd.read_csv('updated_test.csv')
dataset = dataset[ dataset['YEAR']>1980 ]
dataset = dataset.dropna()
X = dataset.iloc[:,[0,3,4,6,7]].values
y = dataset.iloc[:,5].values
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
X[:, 1] = labelencoder_X.fit_transform(X[:, 1])
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])
onehotencoder = OneHotEncoder(categorical_features = [0,1,3])
X = onehotencoder.fit_transform(X).toarray()
X = X[:,1:]
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 0)
onehotencoder = OneHotEncoder(categorical_features = [0])
y_train = np.reshape(y_train,(-1,1))
y_train = onehotencoder.fit_transform(y_train).toarray()
# Feature Scaling
from sklearn.preprocessing import StandardScaler
#normalization
#val-mean/n
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'xyz_models/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# fit standalone model
	model = svm.SVC(kernel='linear')
	model.fit(stackedX, inputy)
	return model

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    print(inputX.shape)
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
# evaluate standalone models on test dataset
for model in members:
    testy_enc = to_categorical(y_test)
    _, acc = model.evaluate(X_test, testy_enc, verbose=0)
    print('Model Accuracy: %.3f' % acc)
# fit stacked model using the ensemble
model = fit_stacked_model(members, X_test, y_test)
# evaluate model on test set
yhat = stacked_prediction(members, model, X_test)
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

>loaded xyz_models/model_1.h5
>loaded xyz_models/model_2.h5
>loaded xyz_models/model_3.h5
>loaded xyz_models/model_4.h5
>loaded xyz_models/model_5.h5
Loaded 5 models
(838,)
(838, 47)
(838, 6)
Model Accuracy: 0.937
(838,)
(838, 47)
(838, 6)
Model Accuracy: 0.937
(838,)
(838, 47)
(838, 6)
Model Accuracy: 0.937
(838,)
(838, 47)
(838, 6)
Model Accuracy: 0.937
(838,)
(838, 47)
(838, 6)
Model Accuracy: 0.937
After train (838, 47)
(838, 47)
Stacked Test Accuracy: 0.937


In [131]:
print(y_test.shape)
print(testy_enc.shape)
print(X_test.shape)
testy_enc = to_categorical(y_test)

(838,)
(838, 6)
(838, 47)


NameError: name 'inputX' is not defined

In [68]:
from sklearn import svm
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 0)
onehotencoder = OneHotEncoder(categorical_features = [0])
#y_train = np.reshape(y_train,(-1,1))
#y_train = onehotencoder.fit_transform(y_train).toarray()

# Feature Scaling
from sklearn.preprocessing import StandardScaler
#normalization
#val-mean/n
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_svm = clf.predict(X_test)
from sklearn import metrics
global svm_score
svm_score=metrics.accuracy_score(y_test, y_pred_svm)
print("SVM ACCURACY: %.3f" % svm_score)
accuracies.append(svm_score)

SVM ACCURACY: 0.942


In [139]:
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
#import tensorflow as tf
dataset = pd.read_csv('updated_test.csv')
print(dataset)



                    SUBDIVISION  YEAR  ANNUAL PRECIPITATION  QUARTER  \
0     ANDAMAN & NICOBAR ISLANDS  1980                3625.3  Jan-Feb   
1     ANDAMAN & NICOBAR ISLANDS  1980                3625.3  Mar-May   
2     ANDAMAN & NICOBAR ISLANDS  1980                3625.3  Jun-Sep   
3     ANDAMAN & NICOBAR ISLANDS  1980                3625.3  Oct-Dec   
4     ANDAMAN & NICOBAR ISLANDS  1981                3074.8  Jan-Feb   
...                         ...   ...                   ...      ...   
4311         WEST UTTAR PRADESH  2014                 486.9  Oct-Dec   
4312         WEST UTTAR PRADESH  2015                 582.7  Jan-Feb   
4313         WEST UTTAR PRADESH  2015                 582.7  Mar-May   
4314         WEST UTTAR PRADESH  2015                 582.7  Jun-Sep   
4315         WEST UTTAR PRADESH  2015                 582.7  Oct-Dec   

      PRECIPITATION  SEVERITY1 TERRAIN   CLIMATE  SEVERITY  
0               6.3          0  Island  Tropical         0  
1            

In [154]:
dataset = dataset[ dataset['YEAR']>1980 ]
dataset = dataset.dropna()
X = dataset.iloc[:,[0,3,4,6,7]].values
y = dataset.iloc[:,5].values
print(X.shape)
print(X[4000][:])
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
X[:, 1] = labelencoder_X.fit_transform(X[:, 1])
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])
print(X[4000][:])
print(X.shape)
onehotencoder = OneHotEncoder(categorical_features = [0,1,3])
X = onehotencoder.transform(X).toarray()
print("\nlength:",len(X[4000][:]))
print(X[4000][:])

X = X[:,1:]
print("\nlength:",len(X[4000][:]))
print(X[4000][:])


(4187, 5)
['WEST RAJASTHAN' 'Mar-May' 10.8 'Desert' 'Hot and dry']
[28 2 10.8 2 6]
(4187, 5)


AttributeError: 'OneHotEncoder' object has no attribute 'feature_indices_'

In [153]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
print("y")
print("length:",len(y))
print(y)
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 0)
onehotencoder = OneHotEncoder(categorical_features = [0])
print('\nX_train')
print(X_train.shape)

print('\nX_test')
print(X_test.shape)

print('\ny_train')
print(y_train.shape)

print('\ny_test')
print(y_test.shape)



y
length: 4187
[0 0 0 ... 0 0 0]

X_train
(3349, 5)

X_test
(838, 5)

y_train
(3349,)

y_test
(838,)


In [142]:
y_train = np.reshape(y_train,(-1,1))
print('\ny_train')
print(y_train.shape)
print(y_train[0])
y_train = onehotencoder.fit_transform(y_train).toarray()
print("\n")
print(y_train.shape)
print(y_train[0])


y_train
(3349, 1)
[0]


(3349, 6)
[1. 0. 0. 0. 0. 0.]


In [143]:

# Feature Scaling
from sklearn.preprocessing import StandardScaler
#normalization
#val-mean/n
print("X_train")
print(X_train[0][:])
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
print("\nNormalization")
print("\nX_train")
print(X_train[0][:])
X_test = sc_X.transform(X_test)
print("\nX_test")
print(X_test[0][:])

X_train
[  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    1.    0.  114.9   3. ]

Normalization

X_train
[-0.18686801 -0.18857274 -0.17270726 -0.1790232  -0.19443768 -0.18167091
 -0.18254602 -0.19277761 -0.18254602 -0.18686801 -0.19026428  5.25584737
 -0.18601059 -0.18167091 -0.18428537 -0.18601059 -0.18772205 -0.18857274
 -0.18514972 -0.18857274 -0.18601059 -0.18428537 -0.18942014 -0.18772205
 -0.18514972 -0.18601059 -0.18254602 -0.18686801 -0.18167091 -0.5671207
 -0.5786145   1.7187074  -0.5818326  -0.50335699 -0.18686801 -0.27345082
 -0.19026428 -0.18254602 -0.18428537 -0.65293206 -0.18772205 -0.18514972
 -0.18341749  2.2608215  -0.18254602 -0.44929246 -1.09172701]

X_test
[-0.18686801 -0.18857274 -0.17270726 -0.1790232  -0.19443768 -0.18167091
 -0.18254602 -0.19277761 -0.182

In [144]:

# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'main_models/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX


In [145]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    print("fit_stacked_model")
    print(inputX.shape)
    print(inputy.shape)
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = svm.SVC(kernel='linear')
    model.fit(stackedX, inputy)
    y_pred_svm = model.predict(i)
    from sklearn import metrics
    svm_score=metrics.accuracy_score(y_test, y_pred_svm)*100
    print("SVM ACCURACY:",svm_score)
    return model


In [146]:
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

In [147]:
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
# evaluate standalone models on test dataset

for model in members:
    testy_enc = to_categorical(y_test)
    print(y_test.shape)
    print(testy_enc.shape)
    _, acc = model.evaluate(X_test, testy_enc, verbose=0)
    print('Model Accuracy: %.3f' % acc)
model = fit_stacked_model(members, X_test, y_test)
print("done F_S_M")
yhat = stacked_prediction(members, model, X_test)
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

>loaded main_models/model_1.h5
>loaded main_models/model_2.h5
>loaded main_models/model_3.h5
>loaded main_models/model_4.h5
>loaded main_models/model_5.h5
Loaded 5 models
(838,)
(838, 6)
Model Accuracy: 0.947
(838,)
(838, 6)
Model Accuracy: 0.942
(838,)
(838, 6)
Model Accuracy: 0.947
(838,)
(838, 6)
Model Accuracy: 0.945
(838,)
(838, 6)
Model Accuracy: 0.945
fit_stacked_model
(838, 47)
(838,)


ValueError: X.shape[1] = 1 should be equal to 30, the number of features at training time

[[   0.     0.     0.  ...    0.    38.9   16. ]
 [   0.     0.     0.  ...    0.   366.7   16. ]
 [   0.     0.     0.  ...    0.  1908.    16. ]
 ...
 [   0.     0.     0.  ...    0.    95.9    3. ]
 [   0.     0.     0.  ...    0.   436.1    3. ]
 [   0.     0.     0.  ...    0.    11.9    3. ]]
